# **Course**: Deep Learning

[<img align="right" width="400" height="100" src="https://www.tu-braunschweig.de/typo3conf/ext/tu_braunschweig/Resources/Public/Images/Logos/tu_braunschweig_logo.svg">](https://www.tu-braunschweig.de/en/)

[Mehdi Maboudi](https://www.tu-braunschweig.de/en/igp/staff/mehdi-maboudi) \([m.maboudi@tu-bs.de](m.maboudi@tu-bs.de)) and [Pedro Achanccaray](https://www.tu-braunschweig.de/en/igp/staff/pedro-diaz) (p.diaz@tu-bs.de)

[Technical University of Braunschweig](https://www.tu-braunschweig.de/en/)  
[Institute of Geodesy and Photogrammetry](https://www.tu-braunschweig.de/igp)

# **Assignment 03:** Vanilla CNN for EuroSAT dataset

## **Load packages and data**

In [1]:
!pip install wandb

In [2]:
!wandb login e0889b0251f131d15192dcbea2f1400c34a44bff

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Disha\.netrc


In [3]:
# Management of files
import os
from os.path import exists, join

# Tensorflow and Keras
from tensorflow.keras.callbacks import ModelCheckpoint, \
                                       EarlyStopping

# Monitor training
import wandb
from wandb.keras import WandbMetricsLogger

# Working with arrays
import numpy as np

# External files with functions to load the dataset,
# create a CNN model, and a data generator.
from importlib import reload
import datasets
import models
import data_generator
# Useful to reload modified external files without need
# of restarting the kernel. Just run again this cell.
reload(datasets)
reload(models)
reload(data_generator)

from datasets import *
from models import *
from data_generator import *

**Variables**

In [4]:
PROJECT_DIR = "." # os.getcwd()
SEED = 42
BATCH_SIZE = 32
TARGET_SIZE = 64

### **Download the dataset**

We will use the [EuroSAT dataset](https://zenodo.org/record/7711810#.ZFn-y3bP1D9) with Sentinel-2 images. There are two versions of this dataset: RGB (3 bands) and MS (multispectral - 13 bands).

For this assignment, we will work with the RGB version. The following lines download the dataset (~130 MB):

In [5]:
url_dataset = "https://zenodo.org/record/7711810/files/EuroSAT_RGB.zip?download=1"
filename = "EuroSAT_RGB.zip"

if not exists("EuroSAT_RGB"):
  !pip install wget
  import wget
  f = wget.download(url_dataset, PROJECT_DIR)
  import zipfile
  with zipfile.ZipFile(filename, "r") as zip_ref:
    zip_ref.extractall(".")
  os.remove(join(PROJECT_DIR, filename))

### **Reading the dataset**

The function **`read_eurosat`** is implemented in the **`datasets.py`** file. The output of this function are a dataframe with information about the image paths and their corresponding classes, and the number of classes.

In [6]:
path_data = join(PROJECT_DIR, "EuroSAT_RGB")

df, n_classes = read_eurosat(path_data=path_data, SEED=SEED)
classes = np.unique(df["class_str"].values)

df

,path_image,class_str,class_int
0,.\EuroSAT_RGB\Forest\Forest_2313.jpg,Forest,1
1,.\EuroSAT_RGB\PermanentCrop\PermanentCrop_2358...,PermanentCrop,6
2,.\EuroSAT_RGB\HerbaceousVegetation\HerbaceousV...,HerbaceousVegetation,2
3,.\EuroSAT_RGB\Pasture\Pasture_1415.jpg,Pasture,5
4,.\EuroSAT_RGB\Highway\Highway_1611.jpg,Highway,3
...,...,...,...
26995,.\EuroSAT_RGB\River\River_76.jpg,River,8
26996,.\EuroSAT_RGB\Forest\Forest_2391.jpg,Forest,1
26997,.\EuroSAT_RGB\AnnualCrop\AnnualCrop_861.jpg,AnnualCrop,0
26998,.\EuroSAT_RGB\Pasture\Pasture_1796.jpg,Pasture,5


### **Train, Validation and Test sets**

Create **three disjoint** sets: `train`, `validation` and `test`.

Use the following proportions:
- `train`: 60%
- `validation`: 20%
- `test`: 20%

Remember to use **stratified sampling** and the given `SEED` for the splits.

For this, **complete the implementation** of the function **`train_val_test_split`** in the file **`datasets.py`**.

_Search for the **`TODO:`** comments in the file._


In [7]:
splits = train_val_test_split(df,
                              val_size=0.2,
                              test_size=0.2,
                              SEED=SEED)

x_train = splits["x_train"]
y_train = splits["y_train"]
x_val = splits["x_val"]
y_val = splits["y_val"]
x_test = splits["x_test"]
y_test = splits["y_test"]

#### **Class distribution**

For **sanity check**, verify the **class distribution** of each set: `train`, `validation` and `test`.

In [8]:
# Number of samples per class
_, counts_train = np.unique(y_train, return_counts=True)
_, counts_val = np.unique(y_val, return_counts=True)
_, counts_test = np.unique(y_test, return_counts=True)

print("Samples per class - train: {}".format(counts_train))
print("Samples per class - val: {}".format(counts_val))
print("Samples per class - test: {}".format(counts_test))

Samples per class - train: [1800 1800 1800 1500 1500 1200 1500 1800 1500 1800]
Samples per class - val: [600 600 600 500 500 400 500 600 500 600]
Samples per class - test: [600 600 600 500 500 400 500 600 500 600]


## **Data generator**

Go to the file **`data_generator.py`** and **complete the implementation** of the data generator.

_Search for the **`TODO:`** comments in the file._

In [9]:
data_gen_train = DataGenerator(path_images=x_train,
                               labels=y_train,
                               batch_size=BATCH_SIZE,
                               n_classes=n_classes,
                               target_size=TARGET_SIZE,
                               shuffle=True)

data_gen_val = DataGenerator(path_images=x_val,
                             labels=y_val,
                             batch_size=BATCH_SIZE,
                             n_classes=n_classes,
                             target_size=TARGET_SIZE,
                             shuffle=False)

# For sanity check, let's see the generator's output
for i, (x, y) in enumerate(data_gen_train):
    print(i, x.shape, y.shape)

0 (32, 64, 64, 3) (32, 10)
1 (32, 64, 64, 3) (32, 10)
2 (32, 64, 64, 3) (32, 10)
3 (32, 64, 64, 3) (32, 10)
4 (32, 64, 64, 3) (32, 10)
5 (32, 64, 64, 3) (32, 10)
6 (32, 64, 64, 3) (32, 10)
7 (32, 64, 64, 3) (32, 10)
8 (32, 64, 64, 3) (32, 10)
9 (32, 64, 64, 3) (32, 10)
10 (32, 64, 64, 3) (32, 10)
11 (32, 64, 64, 3) (32, 10)
12 (32, 64, 64, 3) (32, 10)
13 (32, 64, 64, 3) (32, 10)
14 (32, 64, 64, 3) (32, 10)
15 (32, 64, 64, 3) (32, 10)
16 (32, 64, 64, 3) (32, 10)
17 (32, 64, 64, 3) (32, 10)
18 (32, 64, 64, 3) (32, 10)
19 (32, 64, 64, 3) (32, 10)
20 (32, 64, 64, 3) (32, 10)
21 (32, 64, 64, 3) (32, 10)
22 (32, 64, 64, 3) (32, 10)
23 (32, 64, 64, 3) (32, 10)
24 (32, 64, 64, 3) (32, 10)
25 (32, 64, 64, 3) (32, 10)
26 (32, 64, 64, 3) (32, 10)
27 (32, 64, 64, 3) (32, 10)
28 (32, 64, 64, 3) (32, 10)
29 (32, 64, 64, 3) (32, 10)
30 (32, 64, 64, 3) (32, 10)
31 (32, 64, 64, 3) (32, 10)
32 (32, 64, 64, 3) (32, 10)
33 (32, 64, 64, 3) (32, 10)
34 (32, 64, 64, 3) (32, 10)
35 (32, 64, 64, 3) (32, 10)
36

290 (32, 64, 64, 3) (32, 10)
291 (32, 64, 64, 3) (32, 10)
292 (32, 64, 64, 3) (32, 10)
293 (32, 64, 64, 3) (32, 10)
294 (32, 64, 64, 3) (32, 10)
295 (32, 64, 64, 3) (32, 10)
296 (32, 64, 64, 3) (32, 10)
297 (32, 64, 64, 3) (32, 10)
298 (32, 64, 64, 3) (32, 10)
299 (32, 64, 64, 3) (32, 10)
300 (32, 64, 64, 3) (32, 10)
301 (32, 64, 64, 3) (32, 10)
302 (32, 64, 64, 3) (32, 10)
303 (32, 64, 64, 3) (32, 10)
304 (32, 64, 64, 3) (32, 10)
305 (32, 64, 64, 3) (32, 10)
306 (32, 64, 64, 3) (32, 10)
307 (32, 64, 64, 3) (32, 10)
308 (32, 64, 64, 3) (32, 10)
309 (32, 64, 64, 3) (32, 10)
310 (32, 64, 64, 3) (32, 10)
311 (32, 64, 64, 3) (32, 10)
312 (32, 64, 64, 3) (32, 10)
313 (32, 64, 64, 3) (32, 10)
314 (32, 64, 64, 3) (32, 10)
315 (32, 64, 64, 3) (32, 10)
316 (32, 64, 64, 3) (32, 10)
317 (32, 64, 64, 3) (32, 10)
318 (32, 64, 64, 3) (32, 10)
319 (32, 64, 64, 3) (32, 10)
320 (32, 64, 64, 3) (32, 10)
321 (32, 64, 64, 3) (32, 10)
322 (32, 64, 64, 3) (32, 10)
323 (32, 64, 64, 3) (32, 10)
324 (32, 64, 6

## **Vanilla CNN for image classification**


Go to the file **`models.py`** and **complete the implementation** of the function **`create_cnn`**.

Feel free **to create any CNN** with the following layers: `Conv2D`, `MaxPooling2D`, `BatchNormalization`, `Dropout`, `Flatten` and `Dense`.

_Search for the **`TODO:`** comments in the file._

Then, create a CNN with a given **number of filters** and **filter size**.

In [10]:
# TODO: Declare the following variables:
#       filters: array with the number of filters used in
#                each convolutional layer
#       k: kernel size
filters = [32,64,128]
k = 3

model = create_cnn(filters=filters,
                   k=k,
                   input_shape=(TARGET_SIZE,TARGET_SIZE,3),
                   n_classes=n_classes)
model.summary()

C:\Users\Disha\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 32, 32, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       2,097,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,194,122 (8.37 MB)

 Trainable params: 2,193,674 (8.37 MB)

 Non-trainable params: 448 (1.75 KB)

### **Model callbacks**


Define the callbacks for training:

In [11]:
cb_autosave = ModelCheckpoint("classification_model.keras",
                              mode="max",
                              save_best_only=True,
                              monitor="val_accuracy",
                              verbose=1)

cb_early_stop = EarlyStopping(patience=20,
                              verbose=1,
                              mode="auto",
                              restore_best_weights="True",
                              monitor="val_accuracy")

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="CNN for image classification",
    name="cnn-classification-euroSAT_RGB",

    # track hyperparameters and run metadata
    config={
    "architecture": "CNN",
    "dataset": "EuroSAT_RGB",
    "bs": BATCH_SIZE
    }
)

cb_wandb = WandbMetricsLogger()

callbacks = [cb_autosave, cb_early_stop, cb_wandb]

wandb: Currently logged in as: dishaghosh-applications (dishaxgh). Use `wandb login --relogin` to force relogin


## **Training the model**

In [12]:
history = model.fit(data_gen_train,
                    epochs=100,
                    validation_data=data_gen_val,
                    callbacks=callbacks
                    )

Epoch 1/100


C:\Users\Disha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.4454 - loss: 1.9813
Epoch 1: val_accuracy improved from -inf to 0.29000, saving model to classification_model.keras


wandb: ERROR Unable to log learning rate.


507/507 ━━━━━━━━━━━━━━━━━━━━ 121s 223ms/step - accuracy: 0.4455 - loss: 1.9803 - val_accuracy: 0.2900 - val_loss: 3.0862
Epoch 2/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.6119 - loss: 1.1168
Epoch 2: val_accuracy improved from 0.29000 to 0.36889, saving model to classification_model.keras
507/507 ━━━━━━━━━━━━━━━━━━━━ 103s 203ms/step - accuracy: 0.6119 - loss: 1.1167 - val_accuracy: 0.3689 - val_loss: 3.9625
Epoch 3/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.6645 - loss: 0.9646
Epoch 3: val_accuracy improved from 0.36889 to 0.75278, saving model to classification_model.keras
507/507 ━━━━━━━━━━━━━━━━━━━━ 103s 203ms/step - accuracy: 0.6646 - loss: 0.9645 - val_accuracy: 0.7528 - val_loss: 0.7110
Epoch 4/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.7084 - loss: 0.8125
Epoch 4: val_accuracy did not improve from 0.75278
507/507 ━━━━━━━━━━━━━━━━━━━━ 102s 201ms/step - accuracy: 0.7084 - loss: 0.8124 - val_accuracy: 0.5102 - val_loss: 1.5729


507/507 ━━━━━━━━━━━━━━━━━━━━ 77s 151ms/step - accuracy: 0.9773 - loss: 0.0783 - val_accuracy: 0.8074 - val_loss: 1.4644
Epoch 50/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.9788 - loss: 0.0755
Epoch 50: val_accuracy improved from 0.89759 to 0.89870, saving model to classification_model.keras
507/507 ━━━━━━━━━━━━━━━━━━━━ 78s 154ms/step - accuracy: 0.9787 - loss: 0.0755 - val_accuracy: 0.8987 - val_loss: 0.5843
Epoch 51/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.9758 - loss: 0.0751
Epoch 51: val_accuracy did not improve from 0.89870
507/507 ━━━━━━━━━━━━━━━━━━━━ 77s 152ms/step - accuracy: 0.9758 - loss: 0.0752 - val_accuracy: 0.8819 - val_loss: 0.6349
Epoch 52/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.9795 - loss: 0.0743
Epoch 52: val_accuracy did not improve from 0.89870
507/507 ━━━━━━━━━━━━━━━━━━━━ 110s 216ms/step - accuracy: 0.9795 - loss: 0.0743 - val_accuracy: 0.8761 - val_loss: 0.8359
Epoch 53/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 

507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.9881 - loss: 0.0472
Epoch 74: val_accuracy did not improve from 0.92056
507/507 ━━━━━━━━━━━━━━━━━━━━ 78s 154ms/step - accuracy: 0.9881 - loss: 0.0473 - val_accuracy: 0.8081 - val_loss: 1.4347
Epoch 75/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9821 - loss: 0.0704
Epoch 75: val_accuracy did not improve from 0.92056
507/507 ━━━━━━━━━━━━━━━━━━━━ 79s 155ms/step - accuracy: 0.9821 - loss: 0.0704 - val_accuracy: 0.8956 - val_loss: 0.9166
Epoch 76/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9865 - loss: 0.0484
Epoch 76: val_accuracy did not improve from 0.92056
507/507 ━━━━━━━━━━━━━━━━━━━━ 79s 155ms/step - accuracy: 0.9865 - loss: 0.0484 - val_accuracy: 0.8361 - val_loss: 1.0653
Epoch 77/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.9857 - loss: 0.0490
Epoch 77: val_accuracy did not improve from 0.92056
507/507 ━━━━━━━━━━━━━━━━━━━━ 79s 156ms/step - accuracy: 0.9857 - loss: 0.0490 - val_a

Link to the report: https://wandb.ai/dishaxgh/CNN%20for%20image%20classification/reports/Vanilla-CNN--Vmlldzo4MTA1NTU2

## **Testing the model**

Test your model and report the accuracies for the train, validation and test sets.

In [17]:
data_gen_test = DataGenerator(path_images=x_test,
                              labels=y_test,
                              batch_size=BATCH_SIZE,
                              n_classes=n_classes,
                              target_size=TARGET_SIZE,
                              shuffle=False)

print("Train:")
scores_train = model.evaluate(data_gen_train)
print("Validation:")
scores_val = model.evaluate(data_gen_val)
print("Test:")
scores_test = model.evaluate(data_gen_test)

Train:
507/507 ━━━━━━━━━━━━━━━━━━━━ 31s 60ms/step - accuracy: 0.9994 - loss: 0.0029
Validation:
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.9213 - loss: 0.5597
Test:
169/169 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.9207 - loss: 0.6159
